In [1]:
import numpy as np
import random
import itertools
from operator import attrgetter
from collections import defaultdict
from collections import deque
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd

import PIL
from jinja2 import Environment
from IPython.display import HTML, display, Math
import os

In [2]:
def display(cards):
    card_files = [[]]
    for card in cards:
        name = card.name
        suit = card.suit.lower()
        if card.value < 11:
            name = card_name_map[name]
        card_files[0].append(os.path.join('cards', '{}_of_{}.png'.format(name, suit)))
                                  
    jinja_template = """
        <table style="border:0px solid black;">
        {% for row in array %}
            <tr style="border:0px solid black;">
            {% for col in row %}
                <td style="background-color:white;
                            border-collapse:collapse;
                            text-align:center;
                            border:0px solid black;">
                    <img border="10" src="{{ col }}" alt="N/A" width="100px">
                </td>
            {% endfor %}
            </tr>
        {% endfor %}
         </table>"""
    HTML_content = Environment().from_string(jinja_template).render(array=card_files)
    return HTML(HTML_content)

In [3]:
def display_rows(rows):
    card_files = []
    for i, row in enumerate(rows):
        card_files.append([])
        for card in row.cards:
            name = card.name
            suit = card.suit.lower()
            if card.value < 11:
                name = card_name_map[name]
            card_files[i].append(os.path.join('cards', '{}_of_{}.png'.format(name, suit)))
                                  
    jinja_template = """
        <table style="border:0px solid black;">
        {% for row in array %}
            <tr style="border:0px solid black;">
            {% for col in row %}
                <td style="background-color:white;
                            border-collapse:collapse;
                            text-align:center;
                            border:0px solid black;">
                    <img border="10" src="{{ col }}" alt="N/A" width="100px">
                </td>
            {% endfor %}
            </tr>
        {% endfor %}
         </table>"""
    HTML_content = Environment().from_string(jinja_template).render(array=card_files)
    return HTML(HTML_content)

In [4]:
suits = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}

card_name_map = {
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5,
            'Six': 6,
            'Seven': 7,
            'Eight': 8,
            'Nine': 9,
            'Ten': 10,
            'Jack': 11, 
            'Queen': 12, 
            'King': 13, 
            'Ace': 14
            }

class Card:
    def __init__(self, name, suit):
        self.suit = suit
        self.name = name
        if self.name == 'Six':
            self.name_plural = 'Sixes'
        else:
            self.name_plural = self.name + 's'
        self.value = card_name_map[self.name]
        if self.value < 10:
            self.abbreviation = str(self.value)
        elif self.value == 10:
            self.abbreviation = 'T'
        else:
            self.abbreviation = self.name[0]
    
    def __str__(self):
        return '{} of {}'.format(self.name, self.suit)
    
    def __eq__(self, other):
        return self.value == other.value

In [5]:
class Deck:
    def __init__(self):
        self.deck = []
        self.card_location_map = {}
        for suit in suits:
            for card_name in card_name_map:
                self.deck.append(Card(card_name, suit))
        self.shuffle()
                
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])
    
    def assign_locations(self):
        self.card_location_map = {}
        for i, card in enumerate(self.deck):
            self.card_location_map[str(card)] = i
    
    def shuffle(self):
        random.shuffle(self.deck)
        self.assign_locations()
    
    def remove(self, card):
        card = self.deck.pop(self.card_location_map[str(card)])
        self.assign_locations()
        return card
    
    def draw(self):
        card = self.deck.pop(0)
        self.assign_locations()
        return card
    
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])

In [6]:
df = pd.read_csv('pre_flop_rank.csv')
pre_flop_hand_ranks = {'{}{}'.format(i[1], i[2]): i[0] for i in zip(list(df["Rank"]), list(df['Cards']), list(df['Type']))}


class PreFlopHand:
    def __init__(self, cards):
        self.cards = cards
        self.max_card = max(cards, key=attrgetter('value'))
        self.min_card = min(cards, key=attrgetter('value'))
        self.suited = self.max_card.suit == self.min_card.suit
        self.connected = self.max_card.value - self.min_card.value == 1 or self.max_card.value == 14 and self.min_card.value == 2
        self.pocket_pair = self.max_card == self.min_card
        if self.suited:
            self.hand_type = 's'
        else:
            self.hand_type = 'o'
        if self.pocket_pair:
            self.hand_type = 'p'
        self.abbreviation = "{}{}{}".format(self.max_card.abbreviation, self.min_card.abbreviation, self.hand_type)
        self.rank = pre_flop_hand_ranks[self.abbreviation]

In [7]:
class Hand:
    def __init__(self, cards):
        self.cards = cards
        self.max_card = max(self.cards, key=attrgetter('value'))
        self.card_suit_map = defaultdict(int)
        self.card_count_map = defaultdict(int)
        self.name = ''
        self.score = 0
        
        for card in self.cards:
            self.card_suit_map[card.suit] += 1
            self.card_count_map[card.value] += 1
    
    def is_royal(self):
        return 14 in self.card_count_map and 13 in self.card_count_map and 12 in self.card_count_map and 11 in self.card_count_map and 10 in self.card_count_map
    
    def is_flush(self):
        return len(self.card_suit_map) == 1
    
    def is_straight(self):
        self.cards.sort(key=lambda card_: card_.value)
        straight_flag = True
        for i in range(1, len(self.cards)):
            if self.cards[i].value - 1 != self.cards[i - 1].value:
                straight_flag = False
        
        if 14 in self.card_count_map and 2 in self.card_count_map and 3 in self.card_count_map and 4 in self.card_count_map and 5 in self.card_count_map:
            straight_flag = True
        
        return straight_flag
    
    def is_quads(self):
        for card in self.card_count_map:
            if self.card_count_map[card] == 4:
                return True
        return False
    
    def is_trips(self):
        for card in self.card_count_map:
            if self.card_count_map[card] == 3:
                return True
        return False
    
    def is_two_pair(self):
        pair_count = 0
        for card in self.card_count_map:
            if self.card_count_map[card] == 2:
                pair_count += 1
        return pair_count == 2
    
    def is_pair(self):
        for card in self.card_count_map:
            if self.card_count_map[card] == 2:
                return True
        return False
        
    def evaluate(self):
        royal = self.is_royal()
        flush = self.is_flush()
        straight = self.is_straight()
        quads = self.is_quads()
        trips = self.is_trips()
        two_pair = self.is_two_pair()
        pair = self.is_pair()
        
        if royal and flush:
            self.name = "Royal Flush"
            self.score = 10
            return
        
        if straight and flush:
            self.name = "Straight Flush"
            self.score = 9
            return
        
        if quads:
            self.name = "Quads"
            self.score = 8
            return
        
        if trips and pair:
            self.name = "Full House"
            self.score = 7
            return
        
        if flush:
            self.name = "Flush"
            self.score = 6
            return
        
        if straight:
            self.name = "Straight"
            self.score = 5
            return
        
        if trips:
            self.name = "Trips"
            self.score = 4
            return
        
        if two_pair:
            self.name = "Two Pair"
            self.score = 3
            return
        
        if pair:
            self.name = "Pair"
            self.score = 2
            return
            
        self.name = "High Card"
        self.score = 1        
    
    def __str__(self):
        cards = ''.join(['{} \n'.format(card) for card in self.cards])
        return '{}\n\n{}'.format(self.name, cards)

        
    def display(self):
        return display(self.cards)
        

In [8]:
class Player:
    def __init__(self, name):
        self.name = name   
        self.pre_flop_hand = None
        self.cards = []
        self.hand = None
        self.possible_hands = []
        self.best_hand = None
        self.stack = 200
        self.position = None
        self.moves = {'Raise', 'Check', 'Fold', 'Call'}
        
    def action(self, table):
#         if self.position == 'Small Blind':
#             table.pot += 10
#             self.stack -= 10
        
#         if self.position == 'Big Blind':
#             table.pot += 20
#             self.stack -= 20

        table.pot += 20
        self.stack -= 20
        
    def __str__(self):
        return '{}{}'.format('Player: {}\n'.format(self.name), ''.join(['{} \n'.format(card) for card in self.cards]))
                                                                                               

In [9]:
class Table:
    def __init__(self):
        self.board_cards = []
        self.burned_cards = []
        self.pot = 0
    
    def display(self):
        return display(self.board_cards)

In [11]:
class Game:
    def __init__(self, players):
        self.players = [Player('{}'.format(i)) for i in range(players)]
        self.deck = Deck()
        self.table = Table()
        self.winner = None
#         self.dealer = self.players[0]
#         self.little_blind = self.players[1]
#         if len(self.players) > 2:
#             self.big_blind = self.players[2]
#         else:
#             self.big_blind = self.players[0]
        self.positions = deque(self.players)
        self.rounds = 0
        
            
    def play(self):
        self.reset_players()
        self.reset_table()
        self.set_positions()
        self.deal()
        action = list(self.positions)[2:] + list(self.positions)[:2]
        for player in self.action:
            player.action(self.table)
#         for player in list(self.positions)[2:] + list(self.positions)[:2]:
#             print(player.name)
#             player.action(self.table)
        
        self.flop()
#         for player in list(self.positions)[2:] + list(self.positions)[:2]:
#             print(player.name)
#             player.action(self.table)
            
        self.turn()
#         for player in list(self.positions)[2:] + list(self.positions)[:2]:
#             print(player.name)
#             player.action(self.table)
        
        self.river()
#         for player in list(self.positions)[2:] + list(self.positions)[:2]:
#             print(player.name)
#             player.action(self.table)
#         print(self.winner.best_hand, self.winner.stack, self.table.pot)
        self.get_winner()
#         print(self.winner.best_hand, self.winner.stack, self.table.pot)
#         print(self.winner.stack)
        
        self.winner.stack += self.table.pot
#         print(self.table.pot)
    
        
    def set_positions(self):
        self.positions.append(self.positions.popleft())
#         self.dealer = self.positions[0]
#         self.positions[0].position = 'Dealer'
        
#         self.small_blind = self.positions[1]
#         self.positions[1].position = 'Small Blind'
        
#         self.big_blind = self.positions[2]
#         self.positions[2].position = 'Big Blind'
#         self.action = self.positions[3]

    def reset_players(self):
        for player in self.positions:
            player.hand = []
            player.cards = []
            player.possible_hands = []
            player.best_hand = None
            player.pre_flop_hand = None
        
        self.winner = None
            
    def reset_table(self):
        self.table.board_cards = []
        self.table.burned_cards = []
        self.table.pot = 0
    
    def deal(self):
        self.deck = Deck()
        for player in self.positions:
            player.hand = []
        for i in range(2):
            for player in self.positions:
                player.cards.append(self.deck.draw())
                if i == 1:
                    player.pre_flop_hand = PreFlopHand(player.cards)
    
    def burn(self):
        self.table.burned_cards.append(self.deck.draw())
        
    def draw(self):
        self.table.board_cards.append(self.deck.draw())
                
    def flop(self):
        self.burn()
        for i in range(3):
            self.draw()
    
    def turn(self):
        self.burn()
        self.draw()
    
    def river(self):
        self.burn()
        self.draw()
        
    def get_possible_hands(self, player):
        cards = player.cards + self.table.board_cards
        hands = []
        for comb in itertools.combinations(cards, 5):
            hand = Hand(list(comb))
            hand.evaluate()
            hands.append(hand)
        player.possible_hands = sorted(hands, key=lambda x: x.score)[::-1]
        return player.possible_hands
        
    def get_best_hand(self, player):
        player.best_hand = player.possible_hands[0]
        return player.best_hand
        
    def get_winner(self):
        for player in self.positions:
            self.get_possible_hands(player)
            self.get_best_hand(player)
        
        self.winner = max(self.positions, key=lambda player: player.best_hand.score)
        return self.winner
    
#     def get_nuts(self, player, table)

    def __str__(self):
        return ''.join(['{}\n'.format(player) for player in self.players])
    
    def display_players(self):
        return display_rows(self.players)
    
    def display_best_hands(self):
        return display_rows([player.best_hand for player in self.players])
    

In [12]:
def simulate(n):
    g = Game(10)
    for i in range(n):
        g.play()

In [24]:
g = Game(5)
g.deal()

for player in g.players:
    print(player.pre_flop_hand.rank)
    
g.display_players()

6
77
105
126
169


,
,
,
,
,


In [25]:
g.flop()
g.table.display()

,,


In [26]:
g.turn()
g.table.display()

,,,


In [27]:
g.river()
g.table.display()

,,,,


In [28]:
winner = g.get_winner()

print(winner.name)

print(winner.best_hand)

winner.best_hand.display()

4
Full House

Two of Clubs 
Two of Hearts 
Two of Diamonds 
Seven of Spades 
Seven of Hearts 



,,,,
